# SLIM Elastic Net Recommender

In [ ]:
def init_kaggle(run_on_kaggle, secret_name):
    if run_on_kaggle:
        from kaggle_secrets import UserSecretsClient
        user_secrets = UserSecretsClient()
        personal_token = user_secrets.get_secret(secret_name)
        
        import subprocess
        subprocess.run(["git", "clone", "https://" + personal_token + "@github.com/alecontuIT/rec_sys_challenge2022.git"])
        
        import os
        os.chdir("./rec_sys_challenge2022")
        
        subprocess.run(["pip", "install", "-r", "requirements.txt"])
        return True
    
    else:
        return False

In [ ]:
kaggle = init_kaggle(False, "recsys_git_token")

In [ ]:
import utils
from scipy.stats import loguniform
from Evaluation.Evaluator import EvaluatorHoldout
from skopt.space import Real, Integer, Categorical
from HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs

In [ ]:
from multiprocessing import cpu_count
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender

class SLIMElasticNetRec(MultiThreadSLIM_SLIMElasticNetRecommender):
    def fit(self, 
            alpha=1.0, 
            l1_ratio=0.1, 
            positive_only=True, 
            topK=100,
            verbose=True, 
            workers=int(cpu_count())):
        super(SLIMElasticNetRec, self).fit(
            alpha=alpha, 
            l1_ratio=l1_ratio, 
            positive_only=positive_only, 
            topK=topK,
            verbose=verbose, 
            workers=workers)
        self.RECOMMENDER_VERSION = "alpha-" + str(alpha) +  "_l1_ratio-" + str(l1_ratio) + "_pos_only-" + str(positive_only) + "_topK-" + str(topK)

In [ ]:
recommender_class = SLIMElasticNetRec
dataset_version = "interactions-all-ones"

In [ ]:
URM_all, URM_train, URM_val, ICM = utils.get_data_global_sample(dataset_version=dataset_version, 
                                                                train_percentage=0.7,
                                                                setSeed=True)

In [ ]:
evaluator_validation = EvaluatorHoldout(URM_val, cutoff_list=[10])

## Hyperparameter search

In [ ]:
hyperparameters_range_dictionary = {
    "topK": Integer(5, 1000),
    "l1_ratio": Real(low = 1e-5, high = 1.0, prior = 'log-uniform'),
    "alpha": Real(low = 1e-3, high = 1.0, prior = 'uniform')
}

In [ ]:
recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [URM_train],     
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = {},
    EARLYSTOPPING_KEYWORD_ARGS = {}
)

In [ ]:
hyper_search = utils.bayesian_search(
    recommender_class, 
    recommender_input_args, 
    hyperparameters_range_dictionary, 
    evaluator_validation,
    dataset_version=dataset_version,
    n_cases=100,
    perc_random_starts=0.3
)

## Best Model

In [ ]:
recommender = utils.fit_best_recommender(recommender_class, URM_all, dataset_version)
utils.submission(recommender, dataset_version)

In [ ]:
utils.save_item_scores(recommender_class, 
                       URM_train, 
                       evaluator_validation.users_to_evaluate, 
                       dataset_version, 
                       fast=True)